# TabPFN


## 概要

**TabPFN（Tabular Prior-Data Fitted Network）** は、ドイツのチューリッヒ工科大学（ETH Zurich）らによって開発された、**事前学習済みのトランスフォーマーモデルによる表形式データ分類器** 。

「**1秒でベイズ最適分類を近似**」というコンセプトで、少数データセットにおける汎用的な高精度分類を実現する。

通常の機械学習はデータをもとに学習するが、TabPFNは**事前学習済みモデルを使うだけで即座に予測**できる点が大きな特徴。



v2も出ている

- v1: [[2207.01848] TabPFN: A Transformer That Solves Small Tabular Classification Problems in a Second](https://arxiv.org/abs/2207.01848)
- v2: [Accurate predictions on small data with a tabular foundation model | Nature](https://www.nature.com/articles/s41586-024-08328-6)





:::{card} テーブルデータでGBDTと同程度の性能をより高速に発揮

176のデータセットでGBDT系とNN系アルゴリズムを比較した [McElfresh et al. (2023). When Do Neural Nets Outperform Boosted Trees on Tabular Data?.](https://arxiv.org/abs/2305.02997) ではTabPFNが高い性能を持ちつつCatBoostよりも高速で、n=3000の少ないデータセットでもよいパフォーマンスを発揮していることを報告している。

:::

## コード例

Github: [PriorLabs/TabPFN: ⚡ TabPFN: Foundation Model for Tabular Data ⚡](https://github.com/PriorLabs/TabPFN)

PyPIからインストールできる

```sh
pip install tabpfn
```

In [2]:
from sklearn.datasets import fetch_openml
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tabpfn import TabPFNRegressor

# Load Boston Housing data
df = fetch_openml(data_id=531, as_frame=True)  # Boston Housing dataset
X = df.data
y = df.target.astype(float)  # Ensure target is float for regression

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize the regressor
regressor = TabPFNRegressor()
regressor.fit(X_train, y_train)

# Predict on the test set
predictions = regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)

tabpfn-v2-regressor.ckpt:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

Mean Squared Error (MSE): 10.672898149858451
R² Score: 0.8684473886974637


## Prior-Data Fitted Networks (PFNs)

[[2112.10510] Transformers Can Do Bayesian Inference](https://arxiv.org/abs/2112.10510) (ICLR2022)

大規模な事前分布を学習させ、事後予測分布（posterior predictive distribution; PPD）を出力するためのTransformerベースのアーキテクチャであるPFNを提案

ガウス過程やBayesian neural networksをMCMCより格段に少ない計算量で近似できる。



## TabPFN

[[2207.01848] TabPFN: A Transformer That Solves Small Tabular Classification Problems in a Second](https://arxiv.org/abs/2207.01848)

### 1. Prior-Data Fitted Network (PFN)

PFNは、**多様なシミュレーションデータから事前に学習**し、「未知の表データ分布に対するベイズ最適分類器」を近似するニューラルネットワーク。  

TabPFNはPFNを表形式データに特化させたアーキテクチャ。


学習の流れ：

1. 人工的に生成した数百万の「仮想データセット」を作成  
2. 各データセットに対して「ベイズ最適分類結果（理論上の最良分類）」を計算  
3. それを模倣するようにトランスフォーマーを訓練  
4. 結果、未知のデータセットでも**ベイズ最適に近い予測**を瞬時に生成できるようになる

### 2. モデル構造
TabPFNは、**トランスフォーマーアーキテクチャ**を基盤にしています。  
各サンプル（行）を「トークン」として処理し、自己注意機構（Self-Attention）によってデータ間関係をモデリングします。

- **入力**: 小規模の表データ（通常は1000件未満）
- **出力**: 各クラスの事後確率分布（Bayesian posterior）
- **特徴**:
  - 数値・カテゴリ混在データを直接扱える
  - 欠損値処理も自動
  - 標準化などの前処理も最小限


### 3. 特徴と利点

| 特徴 | 説明 |
|------|------|
| **学習不要** | 事前学習済みモデルを使うため、訓練不要で推論のみ |
| **ベイズ的解釈** | 出力は事後確率を近似し、ベイズ分類器に相当 |
| **超高速推論** | 小規模データなら数ミリ秒〜数秒で完了 |
| **ハイパーパラメータ不要** | モデル選択・学習率・ツリー深度などの調整不要 |
| **精度** | 多くのUCIデータセットでLightGBMやCatBoostに匹敵または上回る |

### 4. 制約・限界

| 制約 | 内容 |
|------|------|
| **データサイズ制限** | 通常は1000件以下・20特徴量以下が推奨 |
| **回帰タスク非対応** | 現行モデルは分類専用（回帰版は研究中） |
| **大規模データ非対応** | メモリ使用量が急増し、GPUが必要 |
| **外挿に弱い** | 訓練分布と異なる構造のデータには不安定 |


## TabPFN v2

## 関連論文



### [[2502.17361] A Closer Look at TabPFN v2: Understanding Its Strengths and Extending Its Capabilities](https://arxiv.org/abs/2502.17361)

目的：

1. TabPFN v2 がなぜ多様なデータに対して高精度を出せるかを理解する
2. TabPFN v2 が苦手とする高次元データ，多クラス，および大規模データに対して，その性能を拡張・改善する手法を提案する

発見：

1. 属性トークン学習を内在化している（Handling heterogeneity）
    - TabPFN v2 では，属性ごとに決定的なトークン（Embedding）をあらかじめ定義するのではなく，推論時にランダムに perturbation を加えた属性トークンを用いる設計になっている（ランダム性を導入） 
    - それにもかかわらず，Transformer の in-context メカニズムを通じて，モデルは属性間の関係を推定し，意味的な特徴を捉える能力を獲得する。すなわち，「トークン学習」を明示的には行わず，推論中に属性関係を学び取っている。 
    - 実験的には，複数回異なるランダム初期化でトークンを与えても，モデルの属性間の attention パターンなどは安定する，という結果が示されている。 
    - → これにより，異なる次元数や属性意味を持つ新しいデータセットにも，チューニングなしで柔軟に適用できるという性質を実現している。
2. 埋め込み器（feature encoder）への転用可能性
    - 通常，TabPFN v2 は in-context learning によって直接ラベルを出力するが，本研究ではこれを利用してインスタンス表現（embedding）を取り出し，その上で線形分類器を載せることで予測する実験を行っている。 
    - 単純な方法（すべての訓練例に対して dummy ラベルで embedding を抽出しようとする方法）は，訓練データとテストデータで表現空間がずれてしまい性能が出ないことが発覚する。 
    - そこで提案されたのが leave-one-fold-out 戦略である。訓練データを複数の折り畳みに分け，ある折りを “クエリ”（dummy ラベル）として embedding を得て，他の折りを “サポート” として学習を保持する。この方式により，訓練とテストで比較可能な embedding を得られるようにする。 
    - 実験では，こうして抽出された embedding に線形モデルを載せても，元の TabPFN v2 の性能に匹敵する結果が出るケースが多く，場合によっては複数層の embedding を連結して性能を上げることも可能であった。 
    - → この結果は，TabPFN v2 が「強力な特徴空間変換器（変換後にクラスを線形分離しやすい空間を作るもの）」として機能していることを示唆する。
3. テスト時の分割・統合法（divide‐and‐conquer）による性能拡張
    - TabPFN v2 には，次のような制限が知られている（元論文でも指摘されていた）：
        1. 次元数（特徴数）が多いデータ
        2. クラス数が 10 を超える多クラス問題
        3. サンプル数（規模）が大きいデータ
    - これらに対して，本論文では以下のような後処理的（post-hoc）アプローチを提案して改善を図っている：

| 問題領域            | 提案手法                                                                                                              | 概要                                                                       |
| --------------- | ----------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------ |
| 高次元             | 特徴のランダムサブセットを複数抽出 → 各サブセットで TabPFN v2 を適用 → 結果をアンサンブル（平均／多数決）                                                     | 特徴数の二次オーダーの計算負荷を抑えつつ性能を維持または改善|
| 多クラス（クラス数 > 10） | 各ラベルを 10 進法表現に分解し，各桁を予測する（digitwise モデル） → 最終的に桁組み合わせでラベルを復元                                                      | 複数の TabPFN v2 モデルを使って多クラス対応を可能にする（およびランダムなマッピングとアンサンブルで安定化） |
| 大規模データ          | ① 訓練セットから 10,000 サンプルをサポートセットとして抽出 → embedding 抽出 → 線形分類器訓練 → アンサンブル <br>② 決定木でデータを部分分割 → 各部分に TabPFN を適用 → 結果の統合 | 再訓練をせずに大規模データに対応可能とする工夫|



### [[2506.08982] On Finetuning Tabular Foundation Models](https://arxiv.org/abs/2506.08982)

Finetuningが不要なことが売りのTabPFNv2だが、Finetuningするとさらに性能が向上することを明らかにした

https://www.themoonlight.io/ja/review/on-finetuning-tabular-foundation-models